## Searching for bouts for a day of ephys recording
- mircophone chan and sync data and bouts are extracted in sglx_preprocess-s_b1253_21-multisess-pouli-ephys

In [2]:
import os
import glob
import socket
import logging
import pickle
import numpy as np
import pandas as pd
from scipy.io import wavfile
from scipy import signal

### Fuck matplotlib, I'm using poltly now
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from importlib import reload

logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter(
        '%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info('Running on {}'.format(socket.gethostname()))

2021-09-26 02:20:54,146 root         INFO     Running on pouli


In [3]:
from ceciestunepipe.file import bcistructure as et
from ceciestunepipe.util.sound import boutsearch as bs

### Get the file locations for a session (day) of recordings

In [4]:
reload(et)
sess_par = {'bird': 's_b1253_21',
           'sess': '2021-07-18',
           'sort': 2}


exp_struct = et.get_exp_struct(sess_par['bird'], sess_par['sess'], ephys_software='sglx')

raw_folder = exp_struct['folders']['sglx']

derived_folder = exp_struct['folders']['derived']

bouts_folder = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx')

sess_bouts_file = os.path.join(bouts_folder, 'bout_sess_auto.pickle')
sess_bouts_curated_file = os.path.join(bouts_folder, 'bout_curated.pickle')

#os.makedirs(bouts_folder, exist_ok=True)

In [5]:
exp_struct['folders']

{'bird': '/mnt/sphere/speech_bci/raw_data/s_b1253_21',
 'raw': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-18',
 'sglx': '/mnt/sphere/speech_bci/raw_data/s_b1253_21/2021-07-18/sglx',
 'kwik': '/experiment/s_b1253_21/sglx/kwik/2021-07-18',
 'processed': '/mnt/sphere/speech_bci/processed_data/s_b1253_21/2021-07-18/sglx',
 'derived': '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx',
 'tmp': '/experiment/tmp/tmp',
 'msort': '/experiment/tmp/s_b1253_21/sglx/msort/2021-07-18',
 'ksort': '/experiment/tmp/s_b1253_21/sglx/ksort/2021-07-18'}

### load concatenated the files of the session

In [8]:
def read_session_auto_bouts(exp_struct):
    # list all files of the session
    # read into list of pandas dataframes and concatenate
    # read the search parameters of the first session
    # return the big pd and the search params
    derived_folder = exp_struct['folders']['derived']
    
    search_params_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_search_params.pickle')
    hparams=None
    with open(search_params_files[0], 'rb') as fh:
        hparams = pickle.load(fh)
    
    sess_bout_pd_file = os.path.join(os.path.split(derived_folder)[0], 'bouts_sglx', 'bout_auto.pickle')
    try:
        bout_pd = pd.read_pickle(sess_bout_pd_file)
    except FileNotFoundError:
        logger.info('Not found session pandas file {}, loading each epoch file and concatenating'.format(sess_bout_pd_file))
        bout_pd_files = et.get_sgl_files_epochs(derived_folder, file_filter='bout_auto.pickle')
        logger.info('Files to load {}'.format(bout_pd_files))
        bout_pd = pd.concat([pd.read_pickle(p) for p in bout_pd_files[:]])
    
    bout_pd.reset_index(inplace=True, drop=True)
    return bout_pd, hparams

bout_pd, hparams = read_session_auto_bouts(exp_struct)

In [9]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      103 non-null    int64  
 1   end_ms        103 non-null    int64  
 2   start_sample  103 non-null    int64  
 3   end_sample    103 non-null    int64  
 4   p_step        103 non-null    object 
 5   rms_p         103 non-null    float64
 6   peak_p        103 non-null    float64
 7   bout_check    103 non-null    bool   
 8   file          103 non-null    object 
 9   len_ms        103 non-null    int64  
 10  syl_in        103 non-null    object 
 11  n_syl         103 non-null    int64  
 12  peaks_p       103 non-null    object 
 13  n_peaks       103 non-null    int64  
 14  l_p_ratio     103 non-null    float64
 15  waveform      103 non-null    object 
 16  confusing     103 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 12.4+ KB


In [7]:
np.unique(bout_pd['start_ms']).shape

(103,)

In [8]:
### list all the source files
sess_wav_files = np.unique(bout_pd['file'].values)
sess_wav_files

array(['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/1615_g0/wav_mic.wav'],
      dtype=object)

#### optional: drop the night recordings

In [9]:
# night_file = sess_wav_files[-1]
# bout_pd.drop(bout_pd[bout_pd['file']==night_file].index, inplace=True)
#print(night_file)

In [10]:
np.unique(bout_pd['file'].values)

array(['/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav',
       '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/1615_g0/wav_mic.wav'],
      dtype=object)

In [11]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_ms      103 non-null    int64  
 1   end_ms        103 non-null    int64  
 2   start_sample  103 non-null    int64  
 3   end_sample    103 non-null    int64  
 4   p_step        103 non-null    object 
 5   rms_p         103 non-null    float64
 6   peak_p        103 non-null    float64
 7   bout_check    103 non-null    bool   
 8   file          103 non-null    object 
 9   len_ms        103 non-null    int64  
 10  syl_in        103 non-null    object 
 11  n_syl         103 non-null    int64  
 12  peaks_p       103 non-null    object 
 13  n_peaks       103 non-null    int64  
 14  l_p_ratio     103 non-null    float64
 15  waveform      103 non-null    object 
 16  confusing     103 non-null    bool   
dtypes: bool(2), float64(3), int64(7), object(5)
memory usage: 12.4+ KB


In [12]:
np.unique(bout_pd['start_ms']).size

103

###### if it wasnt saved (which is a bad mistake), read the sampling rate from the first file in the session

In [13]:
def sample_rate_from_wav(wav_path):
    x, sample_rate = wavfile.read(wav_path)
    return sample_rate

if hparams['sample_rate'] is None:
    one_wav_path = bpd.loc[0, 'file']
    logger.info('Sample rate not saved in parameters dict, searching it in ' + one_wav_path)
    hparams['sample_rate'] = sample_rate_from_wav(one_wav_path)

In [14]:
def cleanup(bout_pd: pd.DataFrame):
    ## check for empty waveforms (how woudld THAT happen???)
    bout_pd['valid_waveform'] = bout_pd['waveform'].apply(lambda x: (False if x.size==0 else True))
    
    # valid is & of all the validated criteria
    bout_pd['valid'] = bout_pd['valid_waveform']
    
    ## fill in the epoch
    bout_pd['epoch'] = bout_pd['file'].apply(lambda x: et.split_path(x)[-2])
    
    # drop not valid and reset index
    bout_pd.drop(bout_pd[bout_pd['valid']==False].index, inplace=True)
    bout_pd.reset_index(drop=True, inplace=True)
    
    # set all to 'confusing' (unchecked) and 'bout_check' false (not a bout)
    bout_pd['confusing'] = True
    bout_pd['bout_check'] = False

cleanup(bout_pd)

In [15]:
bout_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   start_ms        103 non-null    int64  
 1   end_ms          103 non-null    int64  
 2   start_sample    103 non-null    int64  
 3   end_sample      103 non-null    int64  
 4   p_step          103 non-null    object 
 5   rms_p           103 non-null    float64
 6   peak_p          103 non-null    float64
 7   bout_check      103 non-null    bool   
 8   file            103 non-null    object 
 9   len_ms          103 non-null    int64  
 10  syl_in          103 non-null    object 
 11  n_syl           103 non-null    int64  
 12  peaks_p         103 non-null    object 
 13  n_peaks         103 non-null    int64  
 14  l_p_ratio       103 non-null    float64
 15  waveform        103 non-null    object 
 16  confusing       103 non-null    bool   
 17  valid_waveform  103 non-null    boo

In [16]:
np.unique(bout_pd['start_ms']).shape

(103,)

In [17]:
bout_pd

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,syl_in,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch
0,2764745,2770765,69118625,69269125,"[137.18250603801323, 166.1364744613515, 46.022...",12.208090,338.931800,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6020,"[[2764745, 2764785], [2764905, 2765070], [2766...",14,"[1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...",49,122.857143,"[-74, -123, -103, -104, -91, -103, -85, -79, -...",True,True,True,0610_g0
1,2821180,2830650,70529500,70766250,"[73.89770606840956, 76.71229477152468, 50.9882...",12.208090,323.989772,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,9470,"[[2821180, 2821220], [2821420, 2821460], [2822...",10,"[1, 49, 87, 123, 131, 141, 168, 197, 210, 218,...",47,201.489362,"[33, 36, 31, 17, 38, 26, 35, 24, 21, 34, 27, 2...",True,True,True,0610_g0
2,2890955,2897000,72273875,72425000,"[5.723650048239091, 20.093294098403362, 40.008...",12.208090,385.427078,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6045,"[[2890955, 2891215], [2891795, 2891830], [2892...",18,"[12, 19, 26, 37, 49, 155, 169, 181, 196, 208, ...",58,104.224138,"[37, 34, 48, 23, 45, 44, 48, 56, 47, 64, 55, 5...",True,True,True,0610_g0
3,3022775,3030175,75569375,75754375,"[10.760223359304039, 14.367538094139961, 38.22...",12.208090,250.017320,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7400,"[[3022775, 3022805], [3023205, 3023240], [3023...",25,"[2, 28, 37, 86, 100, 111, 137, 144, 157, 173, ...",64,115.625000,"[-105, -115, -130, -129, -143, -135, -149, -14...",True,True,True,0610_g0
4,3052560,3060680,76314000,76517000,"[84.90921288692883, 71.8622817106007, 22.56276...",12.208090,349.395052,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,8120,"[[3052560, 3052595], [3052925, 3052965], [3053...",23,"[4, 14, 73, 89, 100, 116, 136, 148, 160, 171, ...",74,109.729730,"[-27, -40, -36, -37, -40, -32, -32, -28, -30, ...",True,True,True,0610_g0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,16338949,16346744,408473725,408668600,"[58.293510923566394, 93.15441877166121, 117.44...",7.974694,391.944523,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7795,"[[16338949, 16338984], [16339434, 16339464], [...",17,"[2, 15, 35, 68, 100, 447, 533, 613, 650, 670, ...",42,185.595238,"[12, 25, 10, -26, -71, -57, -49, 2, 0, 18, 7, ...",True,True,True,1615_g0
99,17019174,17024594,425479350,425614850,"[11.745667658709257, 36.31338266186312, 2.1468...",4.984864,375.071276,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5420,"[[17019174, 17019209], [17021104, 17021134], [...",18,"[6, 14, 162, 174, 182, 203, 225, 243, 351, 365...",45,120.444444,"[8, 8, 9, 24, 35, 22, 15, 18, 23, 32, 30, 31, ...",True,True,True,1615_g0
100,17509374,17515384,437734350,437884600,"[26.152463844678746, 40.014285626431686, 24.02...",4.984864,173.840210,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6010,"[[17509374, 17509404], [17509729, 17509759], [...",13,"[1, 63, 73, 85, 92, 124, 216, 379, 411, 423, 4...",42,143.095238,"[-23, -20, -54, -28, -32, -14, 26, -10, 2, 14,...",True,True,True,1615_g0
101,17661039,17666114,441525975,441652850,"[126.41149153578236, 49.31676378276574, 20.107...",4.984864,250.969216,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,5075,"[[17661039, 17661079], [17661229, 17661284], [...",14,"[3, 26, 39, 58, 66, 75, 381, 406, 483, 535, 54...",37,137.162162,"[169, 169, 147, 164, 138, 164, 182, 162, 163, ...",True,True,True,1615_g0


In [18]:
reload(et)

<module 'ceciestunepipe.file.bcistructure' from '/home/finch/repos/ceciestunepipe/ceciestunepipe/file/bcistructure.py'>

#### compute the spectrograms

In [19]:
# but don't compute if they were already computed
# try:
#     bout_pd = pd.read_pickle(sess_bouts_file) ## make it jump if there is no sess_bouts file
#     one_spec = bout_pd['spectrogram'][0] ## make it jump if there is no spectrogram key
#     logger.info('loaded bout pandas with spectrograms from ' + sess_bouts_file)
# except KeyError:
#     logger.info('Computing spectrograms')
#     bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
#     logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
#     bout_pd.to_pickle(sess_bouts_file)

In [20]:
# just compute them alright?
logger.info('Computing spectrograms')
bout_pd['spectrogram'] = bout_pd['waveform'].apply(lambda x: bs.gimmepower(x, hparams)[2])
logger.info('saving bout pandas with spectrogram to ' + sess_bouts_file)
bout_pd.to_pickle(sess_bouts_file)

2021-09-22 18:21:05,623 root         INFO     Computing spectrograms
2021-09-22 18:21:44,119 root         INFO     saving bout pandas with spectrogram to /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_sess_auto.pickle


In [21]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,2764745,2770765,69118625,69269125,"[137.18250603801323, 166.1364744613515, 46.022...",12.20809,338.931800,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6020,...,14,"[1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...",49,122.857143,"[-74, -123, -103, -104, -91, -103, -85, -79, -...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2821180,2830650,70529500,70766250,"[73.89770606840956, 76.71229477152468, 50.9882...",12.20809,323.989772,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,9470,...,10,"[1, 49, 87, 123, 131, 141, 168, 197, 210, 218,...",47,201.489362,"[33, 36, 31, 17, 38, 26, 35, 24, 21, 34, 27, 2...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [22]:
bout_pd['file'][0]

'/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav'

## inspect the bouts and curate them

#### visualize one bout

In [23]:
bout_pd.iloc[0]

start_ms                                                    2764745
end_ms                                                      2770765
start_sample                                               69118625
end_sample                                                 69269125
p_step            [137.18250603801323, 166.1364744613515, 46.022...
rms_p                                                      12.20809
peak_p                                                     338.9318
bout_check                                                    False
file              /mnt/sphere/speech_bci/derived_data/s_b1253_21...
len_ms                                                         6020
syl_in            [[2764745, 2764785], [2764905, 2765070], [2766...
n_syl                                                            14
peaks_p           [1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...
n_peaks                                                          49
l_p_ratio                                       

In [24]:
import plotly.express as px
import plotly.graph_objects as go

from ipywidgets import widgets

In [25]:
def viz_one_bout(df: pd.Series, sub_sample=1):
    # get the power and the spectrogram
    sxx = df['spectrogram'][:, ::sub_sample]
    x = df['waveform'][::sub_sample]
    
    # the trace
    tr_waveform = go.Scatter(y=x)
    figwidg_waveform = go.FigureWidget(data=[tr_waveform],
                                      layout= {'height': 300,'width':1000})

    # the spectrogram
    fig_spectrogram = px.imshow(sxx, 
                                     labels={}, 
                                     color_continuous_scale='Inferno',
                                    aspect='auto')

    fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
    fig_spectrogram.update_xaxes(showticklabels=False)
    fig_spectrogram.update_yaxes(showticklabels=False)
    
    
    figwidg_spectrogram = go.FigureWidget(fig_spectrogram)
    
    display(widgets.VBox([figwidg_waveform,
              figwidg_spectrogram]))
    

viz_one_bout(bout_pd.iloc[24])

    'data': [{'type': 'scatter',
              'uid': '1dc55c48-a211-4d5d-bebc-a…

In [26]:
bout_pd.head(2)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,2764745,2770765,69118625,69269125,"[137.18250603801323, 166.1364744613515, 46.022...",12.20809,338.931800,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6020,...,14,"[1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...",49,122.857143,"[-74, -123, -103, -104, -91, -103, -85, -79, -...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2821180,2830650,70529500,70766250,"[73.89770606840956, 76.71229477152468, 50.9882...",12.20809,323.989772,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,9470,...,10,"[1, 49, 87, 123, 131, 141, 168, 197, 210, 218,...",47,201.489362,"[33, 36, 31, 17, 38, 26, 35, 24, 21, 34, 27, 2...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


## use it in a widget


#### add a 'confusing' label, for not/sure/mixed.
we want to avoid having things we are not sure of in the training dataset

In [27]:
bout_pd.reset_index(drop=True, inplace=True)

In [28]:
## Set confusing by default, will only be False once asserted bout/or not
bout_pd['confusing'] = True
bout_pd['bout_check'] = False

In [29]:
### Create a counter object (count goes 1:1 to DataFrame index)
from traitlets import CInt, link

class Counter(widgets.DOMWidget):
    value = CInt(0)
    value.tag(sync=True)

In [30]:
class VizBout():
    def __init__(self, hparams, bouts_pd):
        self.bout = None
        self.bouts_pd = bouts_pd
        self.bout_series = None
        self.is_bout = None
        self.is_confusing = None
        
        self.bout_counter = None
        self.bout_id = None
        
        self.buttons = {}
        self.m_pick = None
        
        
        self.fig_waveform = None
        self.fig_spectrogram = None
        
        self.figwidg_waveform = None
        self.figwidg_spectrogram = None
        
        self.fig_width = 2
        self.sub_sample = 10
        
        self.x = None
        self.sxx = None
        self.tr_waveform = None
        
        self.s_f = hparams['sample_rate']
        
        self.init_fig()
        self.init_widget()
        self.show()
        
    def init_fig(self):
        # the trace
        self.tr_waveform = go.Scatter(y=np.zeros(500))
        self.figwidg_waveform = go.FigureWidget(data=[self.tr_waveform],
                                               layout={'width': 1000, 'height':300})
        
        # the spectrogram
        self.fig_spectrogram = px.imshow(np.random.rand(500, 500), 
                                         labels={}, 
                                         color_continuous_scale='Inferno',
                                        aspect='auto')
        
        self.fig_spectrogram.update_layout(width=1000, height=300, coloraxis_showscale=False)
        self.fig_spectrogram.update_xaxes(showticklabels=False)
        self.fig_spectrogram.update_yaxes(showticklabels=False)
        self.figwidg_spectrogram = go.FigureWidget(self.fig_spectrogram)
        
        
    def init_widget(self):
        # declare elements
        # lay them out
        #
        
        self.bout_counter = Counter()
        self.is_bout = widgets.Checkbox(description='is bout')
        self.is_confusing = widgets.Checkbox(description='Not sure or mixed')
        
        self.buttons['Next'] = widgets.Button(description="Next", button_style='info',
                                              icon='plus')   
        self.buttons['Prev'] = widgets.Button(description="Prev", button_style='warning',
                                              icon='minus')
        self.buttons['Check'] = widgets.Button(description="Check", button_style='success', 
                                               icon='check')
        self.buttons['Uncheck'] = widgets.Button(description="Uncheck", button_style='danger',
                                                 icon='wrong')
        
        [b.on_click(self.button_click) for b in self.buttons.values()]
        
        left_box = widgets.VBox([self.buttons['Prev'], self.buttons['Uncheck']])
        right_box = widgets.VBox([self.buttons['Next'], self.buttons['Check']])
        button_box = widgets.HBox([left_box, right_box])

        self.m_pick = widgets.IntSlider(value=0, min=0, max=self.bouts_pd.index.size-1,step=1, 
                                        description="Bout candidate index")
        
        
        control_box = widgets.HBox([button_box,
                                  widgets.VBox([self.is_bout, self.is_confusing]),
                                    self.m_pick])
        
        link((self.m_pick, 'value'), (self.bout_counter, 'value'))

        self.update_bout()
        
        self.is_bout.observe(self.bout_checked, names='value')
        self.is_confusing.observe(self.confusing_checked, names='value')
        self.m_pick.observe(self.slider_change, names='value')
        
        all_containers = widgets.VBox([control_box, 
                                       self.figwidg_waveform, self.figwidg_spectrogram])
        display(all_containers)
#         display(button_box)
#         display(self.m_pick)
#         display(self.is_bout)
#         display(self.fig)
        
    def button_click(self, button):        
        self.bout_id = self.bout_counter.value
        curr_bout = self.bout_counter
        
        if button.description == 'Next':
            curr_bout.value += 1
        elif button.description == 'Prev':
            curr_bout.value -= 1
        elif button.description == 'Check':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = True
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        elif button.description == 'Uncheck':
            self.bouts_pd.loc[self.bout_id, 'bout_check'] = False
            self.bouts_pd.loc[self.bout_id, 'confusing'] = False
            curr_bout.value += 1
        
        # handle the edges of the counter
        if curr_bout.value > self.m_pick.max:
            curr_bout.value = 0
            
        if curr_bout.value < self.m_pick.min:
            curr_bout.value = self.m_pick.max
    
    def slider_change(self, change):
        #logger.info('slider changed')
        #self.bout_counter = change.new
        #clear_output(True)
        self.update_bout()
        self.show()
            
    def bout_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'bout_check'] = bc['new']
    
    def confusing_checked(self, bc):
#         print "bout checked"
#         print bc['new']
#         print self.motiff
        self.bouts_pd.loc[self.bout_id, 'confusing'] = bc['new']
    
    def update_bout(self):
        self.bout_id = self.bout_counter.value
        self.bout_series = self.bouts_pd.iloc[self.bout_id]
        
        self.is_bout.value = bool(self.bout_series['bout_check'])
        self.is_confusing.value = bool(self.bout_series['confusing'])
        
        self.x = self.bout_series['waveform'][::self.sub_sample]
        self.sxx = self.bout_series['spectrogram'][::self.sub_sample]
        
    def show(self):
        #self.fig.clf()
        #self.init_fig()
        # update
#         self.update_bout()
        #plot
        #logger.info('showing')
        
        # Show the figures
        with self.figwidg_waveform.batch_update():
            self.figwidg_waveform.data[0].y = self.x
            self.figwidg_waveform.data[0].x = np.arange(self.x.size) * self.sub_sample / self.s_f 
            
        with self.figwidg_spectrogram.batch_update():
            self.figwidg_spectrogram.data[0].z = np.sqrt(self.sxx[::-1])
            
        

viz_bout = VizBout(hparams, bout_pd)

### save it

In [31]:
hparams

{'num_freq': 1024,
 'preemphasis': 0.97,
 'frame_shift_ms': 5,
 'frame_length_ms': 10,
 'min_level_db': -55,
 'ref_level_db': 110,
 'mel_filter': False,
 'num_mels': 1024,
 'fmin': 500,
 'fmax': 12000,
 'max_iters': 200,
 'griffin_lim_iters': 20,
 'power': 1.5,
 'read_wav_fun': 'read_wav_chan',
 'file_order_fun': 'sess_file_id',
 'min_segment': 30,
 'min_silence': 2000,
 'min_bout': 5000,
 'peak_thresh_rms': 0.55,
 'thresh_rms': 0.25,
 'mean_syl_rms_thresh': 0.3,
 'max_bout': 120000,
 'l_p_r_thresh': 100,
 'waveform_edges': 1000,
 'bout_auto_file': 'bout_auto.pickle',
 'bout_curated_file': 'bout_checked.pickle',
 'sample_rate': 25000}

In [32]:
### get the curated file path
##save to the curated file path
viz_bout.bouts_pd.to_pickle(sess_bouts_curated_file)
logger.info('saved curated bout pandas to pickle {}'.format(sess_bouts_curated_file))

2021-09-22 18:33:46,940 root         INFO     saved curated bout pandas to pickle /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_curated.pickle


### summary of where the good bouts are along the session

In [50]:
def give_summary(bpd: pd.DataFrame):
    all_files = np.unique(bpd['file'])
    summary_dict = {f: np.where((bpd['bout_check']==True) & (bpd['file']==f))[0].size for f in all_files}
    return summary_dict

sum_dict = give_summary(viz_bout.bouts_pd)
sum_dict

{'/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/0610_g0/wav_mic.wav': 34,
 '/mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/1615_g0/wav_mic.wav': 8}

## Some helper functions for somewhere down the line

In [54]:
sess_par_e = dict.copy(sess_par)
sess_par_e['epoch'] = '0610_g0' 

In [61]:
bpd.head()

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_syl,peaks_p,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram
0,2764745,2770765,69118625,69269125,"[137.18250603801323, 166.1364744613515, 46.022...",12.20809,338.931800,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6020,...,14,"[1, 25, 35, 48, 57, 75, 275, 285, 349, 358, 37...",49,122.857143,"[-74, -123, -103, -104, -91, -103, -85, -79, -...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,2821180,2830650,70529500,70766250,"[73.89770606840956, 76.71229477152468, 50.9882...",12.20809,323.989772,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,9470,...,10,"[1, 49, 87, 123, 131, 141, 168, 197, 210, 218,...",47,201.489362,"[33, 36, 31, 17, 38, 26, 35, 24, 21, 34, 27, 2...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2890955,2897000,72273875,72425000,"[5.723650048239091, 20.093294098403362, 40.008...",12.20809,385.427078,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,6045,...,18,"[12, 19, 26, 37, 49, 155, 169, 181, 196, 208, ...",58,104.224138,"[37, 34, 48, 23, 45, 44, 48, 56, 47, 64, 55, 5...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3022775,3030175,75569375,75754375,"[10.760223359304039, 14.367538094139961, 38.22...",12.20809,250.017320,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,7400,...,25,"[2, 28, 37, 86, 100, 111, 137, 144, 157, 173, ...",64,115.625000,"[-105, -115, -130, -129, -143, -135, -149, -14...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,3052560,3060680,76314000,76517000,"[84.90921288692883, 71.8622817106007, 22.56276...",12.20809,349.395052,False,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,8120,...,23,"[4, 14, 73, 89, 100, 116, 136, 148, 160, 171, ...",74,109.729730,"[-27, -40, -36, -37, -40, -32, -32, -28, -30, ...",True,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [92]:
reload(et)
bpd = et.get_epoch_bout_pd(sess_par_e, only_curated=True)
bpd.head()

2021-09-24 18:39:39,171 ceciestunepipe.file.bcistructure INFO     loading curated bouts for session 2021-07-18 from /mnt/sphere/speech_bci/derived_data/s_b1253_21/2021-07-18/sglx/bouts_ceciestunepipe/bout_curated.pickle
2021-09-24 18:39:42,038 ceciestunepipe.file.bcistructure INFO     Filtering bouts for epoch 0610_g0
2021-09-24 18:39:42,196 ceciestunepipe.file.bcistructure INFO     Filtering also only manually curated bouts


In [94]:
bpd.head(1)

,start_ms,end_ms,start_sample,end_sample,p_step,rms_p,peak_p,bout_check,file,len_ms,...,n_peaks,l_p_ratio,waveform,confusing,valid_waveform,valid,epoch,spectrogram,bird,sess
0,6300478,6351243,157511950,158781075,"[7.886206128106227, 15.074117306094267, 18.369...",22.351748,553.969334,True,/mnt/sphere/speech_bci/derived_data/s_b1253_21...,50765,...,829,61.236429,"[13, -7, 0, 9, -8, 2, -7, -10, -4, -13, -1, 2,...",False,True,True,0610_g0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",s_b1253_21,2021-07-18


In [96]:
bpd['start_ms'].values

array([ 6300478,  7102501,  7180616,  7245376,  7661606,  8031581,
        8128521,  8202321,  8442876,  9853004, 11014954, 12578659,
       14212972, 15410472, 15496242, 17104694, 18395779, 20073717,
       20107052, 21557542, 21939152, 23513780, 23676120, 23753090,
       23766770, 24457160, 24816190, 27239488, 27308728, 27405768,
       27495628, 27737218, 28437448, 29355223])

In [97]:
bpd['end_ms'].values - bpd['start_ms'].values

array([50765, 38435, 31900, 44620, 55725, 47565, 39805, 34070, 60000,
       45875, 45460, 41365, 39195, 58425, 52140, 46640, 21695, 17460,
       39010, 47655, 52790, 23410, 42345,  9165, 18225, 51855, 57915,
       34420, 42935, 35810, 40380, 51640, 63155, 19065])

In [1]:
bpd

NameError: name 'bpd' is not defined